In [82]:
import numpy as np
from sympy import *
    
# Tropical addition
# num - num
# n-vector - n-vector
# nm-matrix - nm-matrix
def tplus(a, b):
    if (mode == 0):
        if (len(np.shape(a)) == 2 and np.shape(a) == np.shape(b)):
            n, m = np.shape(a)
            out = np.zeros(shape=(n,m))
            
            for i in range(n):
                for j in range(m):
                    out[i][j] = min(a[i][j], b[i][j])
            return out
        
        elif (len(np.shape(a)) == len(np.shape(b)) == 1):
            n = np.shape(a)[0]
            out = np.zeros(n)
            
            for i in range(n):
                out[i] = min(a[i], b[i])
            return out
        return min(a, b)
    else:
        if (len(np.shape(a)) == 2 and np.shape(a) == np.shape(b)):
            n, m = np.shape(a)
            out = np.zeros(shape=(n,m))
            
            for i in range(n):
                for j in range(m):
                    out[i][j] = max(a[i][j], b[i][j])
            return out
        
        elif (len(np.shape(a)) == len(np.shape(b)) == 1):
            n = np.shape(a)[0]
            out = np.zeros(n)
            for i in range(n):
                out[i] = max(a[i], b[i])
            return out
        return max(a, b)

# Tropical multiplication
# num - num
# scalar - vector
# scalar - matrix
# nm-matrix - mx-matrix
# nm-matrix - m-vector
def ttimes(a, b):
    if (len(np.shape(a)) == 2 and np.shape(a)[1] == np.shape(b)[0]):
        n, m = np.shape(a)
        if(len(np.shape(b))==1):
            b = b.reshape(len(b),1)
        m, x = np.shape(b)
        out = np.zeros(shape=(n,x))
        
        for i in range(n):
            for j in range(x):
                val = ttimes(a[i][0],b[0][j])
                
                for k in range(m):
                    val = tplus(val, ttimes(a[i][k],b[k][j]))
                out[i][j] = val
        return out
    
    elif (len(np.shape(b)) == 1):
        m = np.shape(b)[0]
        out = np.zeros(m)
        
        for i in range(m):
            out[i] = ttimes(b[i],a)
        return out
    
    elif (len(np.shape(b)) == 2):
        n, m = np.shape(b)
        out = np.zeros(shape=(n, m))
        
        for i in range(n):
            for j in range(m):
                out[i][j] = ttimes(b[i][j], a)
        return out
    
    return (a + b) if (mode2==0) else (a * b)

# Tropical dot product
# n-vector - n-vector
def tdot(a, b):
    val = ttimes(a[0], b[0])
    for i in range(len(a)):
        if mode == 0:
            val = min(val, ttimes(a[i],b[i]))
        else:
            val = max(val, ttimes(a[i],b[i]))
    return val

# Tropical exponentiation
def texp(a, b):
    return (a * b) if (mode2==0) else (a**b)

# Calculate tropical eigenvalue of a square matrix
def teigval(matrix):
    n, m = np.shape(matrix)
    assert n == m
    ma = matrix
    cur = matrix
    for i in range(1,n):
        ma = ttimes(ma, matrix)
        cur = tplus(cur, (1/(i+1))*ma)
    val = cur[0][0]
    for i in range(n):
        val = tplus(val, cur[i][i])
    return val

# Calculate tropical eigenvectors of a square matrix
def teigvec(matrix):
    n, m = np.shape(matrix)
    assert n == m
    
    eigval = teigval(matrix)
    matrix = ttimes(texp(eigval,-1), matrix)
    
    ma = matrix
    cur = matrix
    
    for i in range(1,n):
        ma = ttimes(ma, matrix)
        cur = tplus(cur, ma)
    out = []
    cur = cur.transpose()
    
    for i in range(n):
        if (cur[i][i] == 0):
            out.append(cur[i])
    return basis(out)

# Calculate basis of given list of vectors
def basis(vectors):
    count = 1
    n = len(vectors)
    out = []
    if (n==0):
        return out
    
    out.append(ttimes((texp(vectors[0][0],-1)),vectors[0]))
    
    for i in range(1,n):
        vectors[i] = ttimes(texp(vectors[i][0],-1),vectors[i])
        matches = 0
        
        for j in range(0,len(out)):
            if ((out[j] == vectors[i]).all()):
                matches = 1
        if (matches == 0):
            out.append(vectors[i])
    return out

# Compute transfer matrix
def transfer_matrix(A, h):
    n, m = np.shape(matrix)
    assert n == m
    
    return numpy.exp(A*(1/h))

# Compute transfer asymptote
def matrix_asymptote(A):
    h = Symbol('h')
    
    n, m = np.shape(A)
    assert n == m
    out = np.zeros(shape=(n, m))
    
    for i in range(n):
        for j in range(m):
            out[i][j] = limit(exp((1/h)*A[i][j]),h,0)
    
    return out
            

In [72]:
# 0 : min
# 1 : max
mode = 1

# 0 : plus
# 1 : times
mode2 = 0

In [73]:
a = np.zeros(3)
b = np.zeros(3)
c = np.zeros(3)
a[0], a[1], a[2] = 0, -5, -1
b[0], b[1], b[2] = -1, 0, -1
c[0], c[1], c[2] = -1, -1, 0

In [74]:
m1 = np.zeros(shape=(3,3))
m2 = np.zeros(shape=(3,3))
m1[0], m1[1], m1[2] = a, b, c
m2[0], m2[1], m2[2] = a, b, c
m2 = m2.transpose()
print(m1)

v1 = np.zeros(shape=(3,1))
v2 = np.zeros(shape=(3,1))
v3 = np.zeros(shape=(3,1))
v1[0], v1[1], v1[2] = 0, -1, -1
v2[0], v2[1], v2[2] = 0, 2, 1
v3[0], v3[1], v3[2] = 0, 0, 1

[[ 0. -5. -1.]
 [-1.  0. -1.]
 [-1. -1.  0.]]


In [75]:
print(teigval(m1))

0.0


In [76]:
print(teigvec(m1))

[array([ 0., -1., -1.]), array([0., 2., 1.]), array([0., 0., 1.])]


In [77]:
print(ttimes(m1, v1))

[[ 0.]
 [-1.]
 [-1.]]


In [85]:
comb = tplus(ttimes(-1.5,teigvec(m1)[1]), teigvec(m1)[0])
print(comb)
print(ttimes(m1, comb))

[ 0.   0.5 -0.5]
[[ 0. ]
 [ 0.5]
 [-0.5]]


In [79]:
asym = matrix_asymptote(m1)
print(asym)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
